In [2]:
from __future__ import print_function
import pandas as pd
import datetime
import time
import re
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.client import SignedJwtAssertionCredentials
import json

In [16]:
SCOPE = ["https://spreadsheets.google.com/feeds"]
SECRETS_FILE = "covidtrackerkey.json"
SPREADSHEET = "Health Screen - Microspec (Responses)"
json_key = json.load(open(SECRETS_FILE))
# Authenticate using the signed key
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    'covidtrackerkey.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)
workbook = gc.open(SPREADSHEET)
# Get the first sheet
sheet = workbook.sheet1
data = pd.DataFrame(sheet.get_all_records())
data = data.set_index("Timestamp")
data.index = pd.to_datetime(data.index)
MasterList=pd.read_csv('MasterList.csv')
record=pd.read_csv('record.csv',index_col=0)


In [17]:
data

,"Answer this question only on the first day of your work week. If this is a Monday, was your temperature taken over the weekend on Saturday and Sunday 100° or higher?",Do you have a persistent cough?,Do you have shortness of breath?,First Name,"I am participating in Microspec’s required COVID-19 Daily Safety Screening program. If any, or all, of my symptoms are consistent with cold, flu, respiratory or gastrointestinal illness (the markers of COVID-19), I agree to contact my physician for follow up and report the results to Human Resource. The responsibility for follow up is mine alone and not those associated with this screening. I hereby release Microspec Corporation (""Microspec"" or the “Company"") and each of its employees and agents from any liability arising from, or in any way connected with, this screening and disclosure form to the fullest extent permitted by applicable law.","In the last 14 days, have you had contact with someone who has tested positive for COVID-19?","In the last 24 hours, have you experienced any of the following symptoms: • Chills/shaking with chills • Loss of taste or smell • Muscle pain • Headache • Sore throat • Intestinal distress with vomiting or diarrhea • Nausea • Fever • Cough • Difficulty breathing • Stuffy nose • Congestion?",In the last 24 hours: • Have you taken a fever reducer (such as Tylenol or Advil) to relieve a fever of 100° or higher? • Have you taken a cough suppressant to relieve a dry cough?,"In the last 7 days, have you, or someone in your household traveled outside of New England or traveled anywhere by air?","In the last 7 days, have you, or someone in your household: • Visited inside your home or someone else's home with anyone outside of your household? • Attended an inside or outside public gathering?",Last Name,Phone Number,"Was your temperature, taken prior to leaving home to come to work today, 100° or HIGHER."
Timestamp,,,,,,,,,,,,,
2021-01-04 00:00:00,No,No,No,Morgan,I Accept,No,No,No,No,No,Barnes,6039216801,
2021-01-04 00:00:00,No,No,No,Louis,I Accept,No,No,No,No,No,Benoit,6037626600,
2021-01-04 00:00:00,No,No,No,Sarah,I Accept,No,No,No,Yes,No,Bird,9784951223,
2021-01-04 00:00:00,No,No,No,Aaron,I Accept,No,No,No,No,No,Bresnick,6038010719,
2021-01-04 00:00:00,No,No,No,David,I Accept,No,No,No,No,No,Bulmer,3212521,
2021-01-04 00:00:00,No,No,No,Pauly,I Accept,No,No,No,No,No,C,7697777,
2021-01-04 00:00:00,No,No,No,Michael,I Accept,No,No,No,No,No,Call,6035477696,
2021-01-04 00:00:00,No,No,No,Nate,I Accept,No,No,No,No,No,Connor,6035627683,
2021-01-04 00:00:00,No,No,No,Laura,I Accept,No,No,No,No,No,Copp,6035470105,


In [18]:
compareday= datetime.datetime(2021, 5, 6)
yesframe=pd.DataFrame(columns=['First Name','Last Name'])
noframe=pd.DataFrame(columns=['First Name','Last Name'])
attendanceframe=pd.DataFrame(columns=['First Name','Last Name'])
for x in data.index:
    day = datetime.datetime(x.year,x.month,x.day)
    if (day== compareday):
            
        temp = {'First Name': data['First Name'][x], "Last Name":data['Last Name'][x]}
        attendanceframe= attendanceframe.append(temp,ignore_index=True)
        
        if(data['I am participating in Microspec’s required COVID-19 Daily Safety Screening program. If any, or all, of my symptoms are consistent with cold, flu, respiratory or gastrointestinal illness (the markers of COVID-19), I agree to contact my physician for follow up and report the results to Human Resource. The responsibility for follow up is mine alone and not those associated with this screening. I hereby release Microspec Corporation ("Microspec" or the “Company") and each of its employees and agents from any liability arising from, or in any way connected with, this screening and disclosure form to the fullest extent permitted by applicable law.'][x]!="I Accept" or
        data['Answer this question only on the first day of your work week. If this is a Monday, was your temperature taken over the weekend on Saturday and Sunday 100° or higher?'][x]=='Yes' or
        data['Do you have a persistent cough?'][x]=='Yes'or
        data['Do you have shortness of breath?'][x]=='Yes'or
        data['In the last 24 hours, have you experienced any of the following symptoms: • Chills/shaking with chills • Loss of taste or smell • Muscle pain • Headache • Sore throat • Intestinal distress with vomiting or diarrhea • Nausea • Fever • Cough • Difficulty breathing • Stuffy nose • Congestion?'][x]=='Yes'or
        data['In the last 24 hours: • Have you taken a fever reducer (such as Tylenol or Advil) to relieve a fever of 100° or higher? • Have you taken a cough suppressant to relieve a dry cough?'][x]=='Yes'or
        data["In the last 7 days, have you, or someone in your household: • Visited inside your home or someone else's home with anyone outside of your household? • Attended an inside or outside public gathering?"][x]=='Yes'or
        data["In the last 7 days, have you, or someone in your household traveled outside of New England or traveled anywhere by air?"][x]=='Yes'or
        data["In the last 14 days, have you had contact with someone who has tested positive for COVID-19?"][x]=='Yes'):
            yesframe= yesframe.append(temp, ignore_index=True)
        else:
            noframe= noframe.append(temp, ignore_index=True)


In [19]:
for x in attendanceframe.index:
    name=str(attendanceframe["First Name"][x])+" "+str(attendanceframe["Last Name"][x])
    name= re.sub(r"\s+", "", name)
    name= name.upper()
    for y in MasterList.index:
        mname=str(MasterList["First Name"][y])+" "+str(MasterList["Last Name"][y])
        mname= re.sub(r"\s+", "", mname)
        mname= mname.upper()
        if(mname=="BRUCECRAIG"):
            MasterList=MasterList.drop(y)
        if(mname==name):            
            MasterList=MasterList.drop(y)
            break

In [20]:
colname=str(compareday.month)+"/"+str(compareday.day)+"/"+str(compareday.year)
nonearr=[]
for x in record.index:
    nonearr.append(None)
record[colname]=nonearr
for x in yesframe.index:
    i=re.sub(r"\s+", "", yesframe["First Name"][x])+" "+re.sub(r"\s+", "", yesframe["Last Name"][x])
    i=i.lower()
    i=i.title()
    record[colname][i]="FOLLOW UP"
for x in noframe.index:
    i=re.sub(r"\s+", "", noframe["First Name"][x])+" "+re.sub(r"\s+", "", noframe["Last Name"][x])
    i=i.lower()
    i=i.title()
    record[colname][i]="all set"
for x in MasterList.index:
    i=re.sub(r"\s+", "", MasterList["First Name"][x])+" "+re.sub(r"\s+", "", MasterList["Last Name"][x])
    i=i.lower()
    i=i.title()
    record[colname][i]="DID NOT COMPLETE"
arrr=[]
for x in record.index:
    arrr.append(record[colname][x])
record[colname]=arrr


In [21]:
record.to_csv(r'record.csv', index = True, header=True)

'keck'

'Keck'